In [ ]:
try:
    # settings colab:
    import google.colab
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Checkpoint Reducción de la dimensionalidad

Manifold learning es usado habitualmente para entender la relación entre datos de alta dimensionalidad. 

Un caso muy común es el análisis de imágenes.

Por ejemplo: un set de imágenes con 1000 pixels cada una puede ser pensado como una colección de puntos en 1000 dimensiones, donde cada valor representa el brillo de cada pixel.


## Dataset

En esta práctica trabajaremos con un [dataset de imágenes de caras](http://vis-www.cs.umass.edu/lfw/). 

El dataset se llama "Labeled Faces in the Wild" y es una base de datos para trabajar con reconcimiento de rostros. 

Contiene alrededor de 13000 imágenes recolectadas de la web (nuestro dataset reducido tiene aprox. 2300). 

Cada cara fue taggeada con el nombre de la persona. 

Utilizaremos técnicas para reducir la dimensionalidad de las imágenes y poder visualizarlas.

## Imports

In [ ]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import joblib
#from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn.manifold import TSNE

## Ejercicio 1 - Carga de datos

**Nota Importante:** el dataset está en almacenado en un objeto Bunch, que es un objeto tipo diccionario al que se accede a sus claves como si fueran atributos. 

Podemos acceder a las imagenes de dos formas, con *faces.data* y con *faces.images*. 

En *face.images*, se mantiene el formato de la imagen, es decir que vamos a tener un array de 2D para cada imagen más una dimensión para almacenar cada foto. En *faces.data* los datos están vectorizados para poder usarlos como input de un modelo. Se rompe la estructura 2D de la foto y tenemos solamente una dimensión para cada imagen.

Usemos `shape` para ver las dimensiones de `faces.data` y de `faces.images`

In [ ]:
#Importamos el dataset 
faces = joblib.load("../Data/faces_p2.dump")

In [ ]:
faces.data.shape

In [ ]:
faces.images.shape

In [ ]:
62*47

Tenemos 2.370 imágenes cada una con 2.914 pixels. 

Podemos pensar estas imágenes como puntos en un espacio de 2.914 dimensiones.

## Ejercicio 2 - Visualización

Usando `matplotlib` hagamos una visualización rápida de 32 imágenesen una grilla de 4 filas y 8 columna para ver con qué estamos trabajando.

Ayuda: 

https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flat.html

https://www.w3schools.com/python/ref_func_enumerate.asp


In [ ]:
# este argumento elimina los ticks de la grilla subplot_kw=dict(xticks=[], yticks=[])
fig, ax = plt.subplots(4, 8, subplot_kw=dict(xticks=[], yticks=[]))
#fig, ax = plt.subplots(4, 8)
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='gray')


In [ ]:
faces.target_names

## Ejercicio 3 - PCA

Nos gustaría poder representar estas imágenes (que están en un espacio 2.914-dimensional) en menor dimensionalidad para poder aprender las relaciones fundamentales entre las imágenes. Una primera forma simple de comenzar es computar sus componentes principales y examinar la razón de varianza explicada, que nos da una idea de cuántos componentes principales se requieren para poder describir estos datos.

**3.1** Usando `StandardScaler` transformar `faces.data`

**3.2** Sobre esos datos transformados calcular las primeras 100 componentes principales

**3.3** Graficar el porcentaje de varianza explicada en función de la cantidad de componenetes

In [ ]:
std_sclr = StandardScaler()
X = std_sclr.fit_transform(faces.data)

model_pca = PCA(100).fit(X)

explained_variance = model_pca.explained_variance_ratio_

#print(explained_variance)

cumulative_explained_variance = np.cumsum(explained_variance)

#print(cumulative_explained_variance)

plt.plot(cumulative_explained_variance)
plt.xlabel('número de componentes')
plt.ylabel('% de varianza explicada');

Vemos que para este dataset alrededor de 100 componentes principales son necesarios para preservar el 90% de la varianza. 

Esto nos dice que, en principio, este dataset que es altamente multidimensional no puede ser descripto con pocos componentes lineales.

Cuando este es el caso, métodos de manifold learning como T-SNE o IsoMap pueden ser de mucha ayuda. 

## Ejercicio 4 - Imágenes ubicadas según las primeras dos componentes de PCA

Comencemos ploteando los datos usando las primeras dos componentes de PCA para ver los resultados que obtenemos usando la función `plot_components`

`plot_components` va a graficar thumbnails de las imágenes en las coordenadas de la proyección, que es el output de una proyección bidimensional de todas las imágenes de input.

In [ ]:
def plot_components(proj, images=None, ax=None, thumb_frac=0.05, cmap='gray'):
    
    fig, ax = plt.subplots(figsize=(10, 10))

    # get axis from plot -  https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.gca.html
    ax = ax or plt.gca()
    
    coord_x = proj[:, 0]
    coord_y = proj[:, 1]
    ax.plot(coord_x, coord_y, '.k')    
    # símbolo . color k (black)
    
    if images is not None:
                
        min_dist_2 = (thumb_frac * max(proj.max(0) - proj.min(0))) ** 2
        shown_images = np.array([2 * proj.max(0)])
        
        for i in range(proj.shape[0]):
            
            dist = np.sum((proj[i] - shown_images) ** 2, 1)
            
            # don't show points that are too close
            if np.min(dist) >= min_dist_2:                        
                
                #https://numpy.org/doc/stable/reference/generated/numpy.vstack.html
                shown_images = np.vstack([shown_images, proj[i]])
            
                # grafica las thumbnails:
                imagebox = offsetbox.AnnotationBbox(offsetbox.OffsetImage(images[i], cmap=cmap), proj[i])
                ax.add_artist(imagebox)
            

In [ ]:
pca_2cp = PCA(n_components=2)
proj_pca = pca_2cp.fit_transform(X)

# list[<start>:<stop>:<step>]
plot_components(proj_pca, images = faces.images[:, ::2, ::2])
#plot_components(proj_iso, images=faces.images[:, :, :])
#plot_components(proj_iso, images=faces.images[:, ::3, ::3])

## Ejercicio 5 - Isomap

Proyectemos ahora los datos en 2 dimensiones usando Isomap y usemos `plot_components` para visualizar los resultados

https://scikit-learn.org/stable/modules/generated/sklearn.manifold.Isomap.html#sklearn.manifold.Isomap

In [ ]:
isomap_2cp = Isomap(n_components=2)
proj_iso = isomap_2cp.fit_transform(faces.data)

plot_components(proj_iso, images=faces.images[:, ::2, ::2])


## Ejercicio 6 - TSNE

Proyectemos ahora los datos en 2 dimensiones usando T-SNE y usemos `plot_components` para visualizar los resultados

https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE

In [ ]:
tsne = TSNE(n_components=2, random_state=42)

proj_tsne = tsne.fit_transform(faces.data)

plot_components(proj_tsne, images=faces.images[:, ::2, ::2])

El resultado es interesante: las primeras dos dimensiones del T-SNE parecen describir dos características globales de las imágenes.
    
  * el brillo parece decrecer de abajo hacia arriba
  * la orientación de la cara parece variar de derecha a izquierda